In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.insert(0,"../src")

In [4]:
import pandas as pd
import numpy as np

from fastai2 import *
from fastai2.text.all import *
import vectorize
import helpers
import transformers
from utils import *
from loss.loss import *

In [5]:
full_df = pd.read_pickle("../data/full_df.pkl")

In [6]:
MAX_VOCAB = None
MAX_SEQ_LENGTH = 5000
full_df['TEXT_PROCESSED'] = vectorize.clean_notes(full_df, 'TEXT')
full_df['ICD9_GRP_LIST'] = full_df.ICD9_GRP.apply(lambda x: re.split(" +", x.strip()))

# Use FastAI AWD LSTM

In [7]:
from fastai2.text.all import *

In [8]:
from transformers import BertConfig, BertForSequenceClassification, BertModel, BertTokenizer
bert_tok = BertTokenizer.from_pretrained(
 "bert-base-uncased",
)

fastai_bert_vocab =L(bert_tok.vocab.keys())

In [9]:
sample_df = full_df.sample(frac=0.3)

In [10]:
item_tfms= [[ColReader('TEXT_PROCESSED'), FastAIBertTokenizer(tokenizer=bert_tok, fill_to_max=False) ]]

# Create datasource & dataloaders 

splits = RandomSplitter()(range_of(sample_df))
dsrc = Datasets(sample_df, tfms=item_tfms, splits=splits, dl_type=LMDataLoader)
dls = dsrc.dataloaders(bs=128, seq_len=120)

In [11]:
# We will first train the language model 
dls.show_batch(max_n=2)

[autoreload of utils failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 434, in superreload
    module = reload(module)
  File "/usr/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 781, in get_code
  File "<frozen importlib._bootstrap_external>", line 741, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "../src/utils.py", line 95
    if self.fill_to_max:
                       ^
SyntaxError: 

,text,text_
0,admission date discharge date date of birth sex m service chief complaint cirrhosis secondary to etoh and hepatocellular carcinoma status post chemo embolization history of present illness the patient is a dg year old male with cirrhosis hepatitis b and c hepatocellular carcinoma status post chemo embolization now presenting for orthotopic liver transplant has been admitted twice previously for potential liver transplants on and both were cancelled due to inappropriate organs feeling well no recent infections no fever chills nausea vomiting shortness of breath chest pain headache dysuria other,admission date discharge date date of birth sex m service chief complaint cirrhosis secondary to etoh and hepatocellular carcinoma status post chemo embolization history of present illness the patient is a dg year old male with cirrhosis hepatitis b and c hepatocellular carcinoma status post chemo embolization now presenting for orthotopic liver transplant has been admitted twice previously for potential liver transplants on and both were cancelled due to inappropriate organs feeling well no recent infections no fever chills nausea vomiting shortness of breath chest pain headache dysuria other associated
1,##g mg tablet chewable dg tablet s by mouth every evening ergocalciferol vitamin ddg vitamin d prescribed by other provider dg dg unit capsule dg capsule s by mouth daily discharge medications patient expired discharge disposition expired discharge diagnosis patient expired discharge condition patient expired discharge instructions patient expired followup instructions patient expired admission date discharge date date of birth sex m service medicine allergies patient recorded as having no known allergies to drugs attending chief complaint acute shortness of breath major surgical or invasive procedure cardiac catheterization percutaneous coronary angioplasty,mg tablet chewable dg tablet s by mouth every evening ergocalciferol vitamin ddg vitamin d prescribed by other provider dg dg unit capsule dg capsule s by mouth daily discharge medications patient expired discharge disposition expired discharge diagnosis patient expired discharge condition patient expired discharge instructions patient expired followup instructions patient expired admission date discharge date date of birth sex m service medicine allergies patient recorded as having no known allergies to drugs attending chief complaint acute shortness of breath major surgical or invasive procedure cardiac catheterization percutaneous coronary angioplasty with


In [12]:
dls.vocab = fastai_bert_vocab

In [13]:
learn = language_model_learner(
    dls, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

RuntimeError: CUDA out of memory. Tried to allocate 42.00 MiB (GPU 0; 15.90 GiB total capacity; 68.54 MiB already allocated; 36.62 MiB free; 74.00 MiB reserved in total by PyTorch)

In [ ]:
learn.lr_find()

# Model training

It takes quite a while to train each epoch, so we'll be saving the intermediate model results during the training process. Since fine_tune doesn't do that for us, we'll just use fit_one_cycle. Just like cnn_learner, language_model_learner automatically calls freeze when using a pretrained model (which is the default), so this will only train the embeddings (which is the only part of the model that contains randomly initialized weights--i.e. embeddings for words that are in our IMDb vocab, but aren't in the pretrained model vocab)

In [ ]:
learn.fit_one_cycle(1, 2e-2)

In [ ]:
learn.save('1epoch')
learn = learn.load('1epoch')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

learn.save_encoder('finetuned')


# Check the language model

In [ ]:
TEXT = "He went to doctor because he is feeling"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

print("\n".join(preds))


# Classifier model

In [ ]:
item_tfms= [[ColReader('TEXT_PROCESSED'), FastAIBertTokenizer(tokenizer=bert_tok, fill_to_max=False) ],
            [ColReader('ICD9_GRP_LIST') , MultiCategorize, OneHotEncode]
           ]

# Create datasource & dataloaders 

splits = RandomSplitter()(range_of(sample_df))
dsrc = Datasets(sample_df, tfms=item_tfms, splits=splits)
dls = dsrc.dataloaders(bs=4)

In [ ]:
b = dls.one_batch()

In [ ]:
dls.show_batch(max_n=3)

In [ ]:
# Metrics
acc_02 = partial(accuracy_multi, thresh=0.2)
f_score = F1ScoreMulti(thresh=0.2)

learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, 
                                metrics=[acc_02, f_score]).to_fp16()

In [ ]:
learn = learn.load_encoder('finetuned')


## Training 

In [ ]:
learn.fit_one_cycle(1, 2e-2)


In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))